In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from datetime import datetime

In [ ]:
def calculate_average_movement(arrows, n=50, scale_factor=30, hip_scale_factor=50, is_hip_center=False):
    if len(arrows) < n:
        return arrows[-1][0], arrows[-1][1], arrows[-1][0], arrows[-1][1]  # データが不足している場合は最新の位置を使用
    # 過去 n フレームの平均 dx, dy を計算
    avg_dx = sum((arrows[i][0] - arrows[i-1][0] for i in range(-1, -n, -1))) / (n-1)
    avg_dy = sum((arrows[i][1] - arrows[i-1][1] for i in range(-1, -n, -1))) / (n-1)
    last_x, last_y = arrows[-1]
    # hip_center用の特別なスケールファクターを適用
    if is_hip_center:
        next_x = last_x + int(avg_dx * hip_scale_factor)
        next_y = last_y + int(avg_dy * hip_scale_factor)
    else:
        next_x = last_x + int(avg_dx * scale_factor)
        next_y = last_y + int(avg_dy * scale_factor)
    return last_x, last_y, next_x, next_y

def draw_smooth_arrows(frame, tracks, red_color, green_color, line_thickness, n=50, scale_factor=30, hip_scale_factor=50):
    for key, data in tracks.items():
        arrows = data['arrows']
        if len(arrows) > 1:
            if key == 'hip_center':
                last_x, last_y, next_x, next_y = calculate_average_movement(arrows, n, scale_factor, hip_scale_factor, is_hip_center=True)
            else:
                last_x, last_y, next_x, next_y = calculate_average_movement(arrows, n, scale_factor)
            arrow_color = green_color if key != 'hip_center' else red_color
            cv2.arrowedLine(frame, (last_x, last_y), (next_x, next_y), arrow_color, line_thickness, tipLength=0.5)


def update_landmark_positions(results, tracks, dot_history_length, arrow_history_length, frame):
    landmarks = {
        'left_wrist': results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST],
        'right_wrist': results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST],
        'left_ankle': results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE],
        'right_ankle': results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]
    }
    # Calculate hip center position
    left_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
    right_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
    hip_center_x = int((left_hip.x + right_hip.x) * 0.5 * frame.shape[1])
    hip_center_y = int((left_hip.y + right_hip.y) * 0.5 * frame.shape[0])
    # Update hip center positions for dot history
    tracks['hip_center']['positions'].append((hip_center_x, hip_center_y))
    if len(tracks['hip_center']['positions']) > dot_history_length:
        tracks['hip_center']['positions'].pop(0)
    # Update hip center arrows for arrow history
    tracks['hip_center']['arrows'].append((hip_center_x, hip_center_y))
    if len(tracks['hip_center']['arrows']) > arrow_history_length:
        tracks['hip_center']['arrows'].pop(0)

    for key, landmark in landmarks.items():
        x = int(landmark.x * frame.shape[1])
        y = int(landmark.y * frame.shape[0])
        # Update positions for dot history
        tracks[key]['positions'].append((x, y))
        if len(tracks[key]['positions']) > dot_history_length:
            tracks[key]['positions'].pop(0)
        # Update arrows for arrow history
        tracks[key]['arrows'].append((x, y))
        if len(tracks[key]['arrows']) > arrow_history_length:
            tracks[key]['arrows'].pop(0)

In [ ]:
# ビデオのパスを直接指定
video_path = r"C:\Users\Hassan\Desktop\ボルダリング\input\IMG_9139.mov"

# モジュールの初期化
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 日付を取得し、フォーマットする
today_date = datetime.now().strftime("%Y%m%d")
print("Today's Date:", today_date)  # デバッグ出力

# 元ファイル名を取得し、新しいファイル名を生成
base_name = video_path.split("\\")[-1].split(".")[0]
output_name = f"{base_name}_{today_date}_arrows.mov"
print("Output File Name:", output_name)  # デバッグ出力

# 色の定義
white_color = (255, 255, 255)
green_color = (0, 255, 0)
red_color = (0, 0, 255)
line_thickness = 10

cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_name, fourcc, 60.0, (int(cap.get(3)), int(cap.get(4))))

dot_history_length = 100
arrow_history_length = 750
tracks = {
    'left_wrist': {'positions': [], 'arrows': []},
    'right_wrist': {'positions': [], 'arrows': []},
    'left_ankle': {'positions': [], 'arrows': []},
    'right_ankle': {'positions': [], 'arrows': []},
    'hip_center': {'positions': [], 'arrows': []}
}

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
         # 関節間の接続線を描画（顔のランドマークを除外）
        for connection in mp_pose.POSE_CONNECTIONS:
            start, end = connection
            # 顔と口のランドマークを除外
            if start not in [mp_pose.PoseLandmark.NOSE, mp_pose.PoseLandmark.RIGHT_EYE_INNER, mp_pose.PoseLandmark.RIGHT_EYE, mp_pose.PoseLandmark.RIGHT_EYE_OUTER, mp_pose.PoseLandmark.LEFT_EYE_INNER, mp_pose.PoseLandmark.LEFT_EYE, mp_pose.PoseLandmark.LEFT_EYE_OUTER, mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.MOUTH_LEFT, mp_pose.PoseLandmark.MOUTH_RIGHT] and \
               end not in [mp_pose.PoseLandmark.NOSE, mp_pose.PoseLandmark.RIGHT_EYE_INNER, mp_pose.PoseLandmark.RIGHT_EYE, mp_pose.PoseLandmark.RIGHT_EYE_OUTER, mp_pose.PoseLandmark.LEFT_EYE_INNER, mp_pose.PoseLandmark.LEFT_EYE, mp_pose.PoseLandmark.LEFT_EYE_OUTER, mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.MOUTH_LEFT, mp_pose.PoseLandmark.MOUTH_RIGHT]:
                cv2.line(frame, 
                         (int(results.pose_landmarks.landmark[start].x * frame.shape[1]), int(results.pose_landmarks.landmark[start].y * frame.shape[0])),
                         (int(results.pose_landmarks.landmark[end].x * frame.shape[1]), int(results.pose_landmarks.landmark[end].y * frame.shape[0])),
                         white_color, 2) # 線の太さ
                
        # 各関節にマーカーを配置
        for id, landmark in enumerate(results.pose_landmarks.landmark):
            landmark_type = mp_pose.PoseLandmark(id).name
            if landmark_type in ["LEFT_SHOULDER", "RIGHT_SHOULDER", "LEFT_ELBOW", "RIGHT_ELBOW", 
                                 "LEFT_WRIST", "RIGHT_WRIST", "LEFT_HIP", "RIGHT_HIP", 
                                 "LEFT_KNEE", "RIGHT_KNEE", "LEFT_ANKLE", "RIGHT_ANKLE"]:
                cv2.circle(frame, (int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])), 7, white_color, -1) # マーカーサイズ

        # 腰の中心位置の計算
        left_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
        right_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
        hip_center_x = int((left_hip.x + right_hip.x) * 0.5 * frame.shape[1])
        hip_center_y = int((left_hip.y + right_hip.y) * 0.5 * frame.shape[0])

        # 肩の中心位置の計算
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        shoulder_center_x = int((left_shoulder.x + right_shoulder.x) * 0.5 * frame.shape[1])
        shoulder_center_y = int((left_shoulder.y + right_shoulder.y) * 0.5 * frame.shape[0])

        # 背骨（肩の中心から腰の中心まで）の描画
        cv2.line(frame, (shoulder_center_x, shoulder_center_y), (hip_center_x, hip_center_y), white_color, 2)# 線の太さ

        # 逆三角形を腰の中心位置に描画
        triangle_size = 20
        triangle_points = np.array([
            (hip_center_x, hip_center_y + triangle_size),  # 下部の点
            (hip_center_x - triangle_size, hip_center_y - triangle_size),  # 左上の点
            (hip_center_x + triangle_size, hip_center_y - triangle_size)  # 右上の点
        ])
        cv2.drawContours(frame, [triangle_points], 0, white_color, -1)

        # フレームを出力
        out.write(frame)

        update_landmark_positions(results, tracks, dot_history_length, arrow_history_length, frame)
        draw_smooth_arrows(frame, tracks, red_color, green_color, line_thickness)

        # フレームを出力
        out.write(frame)

    out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()